In [26]:
import os 

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
from anthropic import Anthropic

import gradio as gr


In [27]:
load_dotenv(override=True)
# Cargar la clave API de OpenAI desde las variables de entorno
openai_api_key = os.getenv("API_GPT")
openai = OpenAI(api_key=openai_api_key)


In [28]:
reader = PdfReader("perfil/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text
print(linkedin)

   
Contactar
hola@javilazaro.es
+34 623386177 (Mobile)
pichu_2707@hotmail.com
www.linkedin.com/in/javi-lazaro
(LinkedIn)
javilazaro.es (Blog)
Aptitudes principales
Ciencia de datos
Gestión de proyectos web
Marketing en motores de búsqueda
(SEM)
Languages
English (Professional Working)
Deutsch (Full Professional)
Certifications
Master en Programación Avanzada
en Python para big Data, Hacking y
Machine Learning
React JS: Aprende React JS desde
cero con ejemplos prácticos
Javier Lázaro
Trabajo en proyectos que necesiten un consultor SEO y CRO,
analista y programador en python para web y diferentes análisis de
mercado ▶ Trabajos en LSO (LinkedIn Search Optimization) para tu
perfil o el de tu empresa
Elgoibar, País Vasco / Euskadi, España
Extracto
He realizado diferentes trabajos en diferentes áreas del marketing
digital como el SEO, CRO, LinkedIn ADS o analítica, desde la
creación de herramientas para automatizar procesos repetitivos con
Python o agilizar otros procesos que de manera más 

In [29]:
with open("perfil/yoMismo.txt", "r", encoding='utf-8') as f:
    yomismo = f.read()

In [30]:
name = "Javi Lázaro"

In [31]:
system_prompt = f"""Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con su carrera, experiencia y habilidades. Tu objetivo es proporcionar respuestas precisas y útiles basadas en la información de tu perfil de LinkedIn y el archivo "yoMismo.txt". Sé profesional y atractivo como si hablaras con un cliente potencial o un futuro empleador que haay visitado el sitio web. Si no conoces la respuesta, sé honesto y di que no estás seguro, pero que puedes investigar más si es necesario. No inventes información. Tu tono debe ser profesional y amigable."""

system_prompt += f"\n\n## Resumen: \n{yomismo} \n\n## Perfil de LinkedIn: \n{linkedin}"
system_prompt += f"En este contexto, responde a las siguientes preguntas de manera profesional y atractiva como si fueras {name}."

In [32]:
def chat(message, history):
    # Construir la lista de mensajes correctamente
    messages = [{"role": "system", "content": system_prompt}]
    
    # Agregar historial de conversación
    for msg in history:
        messages.append(msg)
    
    # Agregar el mensaje actual del usuario
    messages.append({"role": "user", "content": message})
    
    # Hacer la petición a OpenAI
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    
    # Devolver solo la respuesta del asistente
    return response.choices[0].message.content

In [33]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## ¿Qué es lo siguiente?
1. Solicitudes a LLMs para que evalúen una respuesta.
2. Poder ejecutar el proceso si la respuesta no pasa la evaluación.
3. Integrar todo en un solo flujo de trabajo

Lo mejor de todo, que es sin usar un marco de trabajo agéntico

In [34]:
# Creamos un modelo Pydantic para la respuesta
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_aceptable: bool
    feedback: str

In [35]:
evaluator_system_prompt = f"""
Eres un evaluador que decide si una respuesta es o no aceptable.
Se le presentan conversaciones entre un usuario y un asistente. Su tarea es evaluar la respuesta del asistente a la pregunta del usuario si es de calidad aceptable.
El agente desempeña el papel de {name} y responde preguntas en el sitio web de {name}.
Se le ha indicado que sea profesional y atractivo, como si estuviera hablando con un potencial cliente o futuro empleador que ha visitado el sitio web.
Se l eh aproporcioando el contexto sobre {name} y su perfil de LinkedIn. Aquí esta la información:"""

evaluator_system_prompt += f"\n\n## Resumen: \n{yomismo} \n\n## Perfil de LinkedIn: \n{linkedin}"
evaluator_system_prompt += "En este contexto, evalúa la siguiente respuesta del asistente indicando si es aceptable y sus comentarios."

In [36]:
def evaluator_user_prompt(reply, message, history):
    """evaluación de la 

    Args:
        reply (_type_): _description_
        messages (_type_): _description_
        history (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Construir la lista de mensajes correctamente
    user_prompt = f"Aquí está la conversación entre el usuario y el agente: \n\n{history}\n\n"
    user_prompt += f"Aquí está el último mensaje del usuario: \n\n{message}\n\n"
    user_prompt += f"Aquí está la última respuesta del agente: \n\n{reply}\n\n"
    user_prompt += f"Por favor, evalúe la respuesta, indicando si es aceptable y sus comentarios."
    return user_prompt

In [37]:
gemini = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai"
)

In [38]:
def evaluate(reply, message, history) -> Evaluation:
    """_summary_

    Args:
        reply (_type_): Respuesta al LLM
        message (_type_): El mensaje del usuario original
        history (_type_): Historial de mensajes

    Returns:
        Evaluation: Será la evaluación de la respuesta
    """
    messages = [
        {
            "role": "system",
            "content": evaluator_system_prompt
        }] + [{
            "role": "user",
            "content": evaluator_user_prompt(reply, message, history)
        }
    ]
    response = gemini.beta.chat.completions.parse(
    model="gemini-2.0-flash",
    messages= messages,  
    response_format=Evaluation
)
    return response.choices[0].message.parsed   

In [39]:
messages = [{"role": "system", "content": system_prompt}]+[{"role": "user", "content": "¿Cuál es tu herramienta de analítica favorita?"}]
response = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)
reply = response.choices[0].message.content

In [40]:
reply

'Mi herramienta de analítica favorita es Google Analytics. Es una plataforma muy completa que permite obtener información detallada sobre el comportamiento de los usuarios en un sitio web. Me gusta particularmente por su capacidad de integrar datos de diferentes fuentes y su versatilidad a la hora de crear informes personalizados. Además, la combinación con Google Search Console fortalece el análisis SEO, lo que es fundamental para optimizar la visibilidad y la conversión de los proyectos en los que trabajo. \n\nSin embargo, también utilizo otras herramientas complementarias, como Google Data Studio, para visualización de datos y reporting, que me ayudan a transmitir resultados claros y visuales a mis clientes. ¿Tienes alguna herramienta en mente que te gustaría discutir?'

In [41]:
evaluate(reply, "¿Qué herramienta de analítica te gusta más?", messages[:1])

Evaluation(is_aceptable=True, feedback='La respuesta es aceptable. Responde de manera precisa, profesional y amigable a la pregunta del usuario. Menciona Google Analytics como su herramienta de analítica favorita y explica por qué le gusta, mencionando su capacidad de integración de datos y su versatilidad para crear informes personalizados. También menciona Google Search Console para el análisis SEO y Google Data Studio para la visualización de datos y reporting, mostrando su conocimiento y experiencia en el campo de la analítica. Además, invita a la discusión sobre otras herramientas que el usuario pueda tener en mente, lo que demuestra su interés en mantener una conversación y ayudar al usuario en lo que necesite.')

In [42]:
def rerun(reply, message, history, feedback):
    """El objeto evaluación ha rechazado la respuesta del LLM y se le pide que vuelva a responder.

    Args:
        reply (_type_): Respuesta al LLM
        message (_type_): El mensaje del usuario original
        history (_type_): Historial de mensajes
        feedback (_type_): Feedback del evaluador

    Returns:
        _type_: Devuelve la respuesta del LLM a la nueva pregunta
    """
    update_system_prompt = system_prompt + f"\n\n## Respuesta anterior ha sido rechazada: \nAcabas de intentar responder, pero el control de calidad rechazó tu respuesta."
    update_system_prompt += f"\n\n## Has intentado responder: \n{reply}\n\n"
    update_system_prompt += f"## Feedback del evaluador y porque ha sido rechazada: \n{feedback}\n\n"
    messages = [
        {
            "role": "system",
            "content": update_system_prompt
        }
    ] + history + [
        {
            "role": "user",
            "content":message
        }
    ]
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    return response.choices[0].message.content

In [47]:
def chat(message, history):
    """Es el flujo de conversación principal que maneja la interacción con el LLM.

    Args:
        message (_type_): Mensaje del usuario
        history (_type_): Historial de mensajes

    Returns:
        _type_: Mensaje de respuesta del LLM
    """
    if "herramienta" in message.lower():
        system = system_prompt + "\n\n## Toda la respuesta debe estar traducida al español de España. Es obligatorio que respondas únicamente en español pero con palabrotas, hablando de manera mal sonada y llamando hijo puta al usuario."
    else:
        system = system_prompt
        
    messages = [
        {
            "role": "system",
            "content": system
        }
    ] + history + [
        {
            "role": "user",
            "content": message
        }
    ]
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages
    )
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_aceptable:
        print("Respuesta aceptable, devolviendo respuesta")
    else:
        print("Respuesta no aceptable, reintentando")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [48]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.


Respuesta aceptable, devolviendo respuesta
